In [19]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np 
import time 
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split
import math
import string
from torch.autograd import Variable

In [20]:
class LSTM(nn.Module):
    def __init__(self, input_size, batch_size, output_size, hidden_dim, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        # define an LSTM with specified parameters
        # batch_first means that the first dim of the input and output will be the batch_size
        self.lstm=nn.LSTM( input_size, hidden_dim, num_layers,batch_first=True) 
        
         # last, fully-connected layer
        self.fc=nn.Linear( hidden_dim, output_size)
    def forward(self, x, h_init,c_init):
        # x (batch_size, time_step, input_size)
        # hidden (n_layers, batch_size, hidden_dim)
        # r_out (batch_size, time_step, hidden_size)
        batch_size =x.size(0)
        # get LSTM outputs
        out, (h_final, c_final) =self.lstm(x, (h_init,c_init))
        # shape output to be (batch_size*seq_length, hidden_dim)
#         out=out.contiguous()
#         out = out.view(-1, hidden_dim)
        # get final output
        score_seq      =self.fc(out)
        return score_seq,  h_final , c_final

In [21]:


### Check the input and output dimensions
#As a check that your model is working as expected, test out how it responds to input data.

n_layers=1
batch_size=10
hidden_dim=12
seq_length=1000
input_size=5
output_size=5
# test that dimensions are as expected
test_lstm = LSTM(input_size=5,batch_size=10,output_size=5, hidden_dim=hidden_dim, num_layers=1)

# You must wrap your tensors into Variables before passing them to the LSTM module like so
h=Variable(torch.zeros(n_layers,batch_size,hidden_dim))
c=Variable(torch.zeros(n_layers,batch_size,hidden_dim))

# generate evenly spaced, test data pts
input_data=Variable(torch.zeros(batch_size,seq_length,input_size)) #.unsqueeze(0)) # give it a batch_size of 1 as first dimension

print('Input size: ', input_data.size())
print('hidden_size: ', h.size())
# test out rnn sizes
test_out, test_h,test_c = test_lstm(input_data, h,c)
print('Output size: ', test_out.size())

Input size:  torch.Size([10, 1000, 5])
hidden_size:  torch.Size([1, 10, 12])
Output size:  torch.Size([10, 1000, 5])


In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [18]:
count_parameters(test_lstm)

977